In [1]:
import torch  
import smplx  
import numpy as np  
import imageio 
import os,sys   
import matplotlib.pyplot as plt 

# data = np.load('/Volumes/Work/Chinese_CV/CCM/Papers/SMPLer-X/demo/results/demo00001/smplx/00002_0.npz')  

/opt/miniconda3/envs/gui/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# orignal demo 
model_folder = f"/Users/lxy/Gitlab/smplx/models/smplx/SMPLX_NEUTRAL.npz"   
model_type = 'smplx'
gender = 'neutral'
use_face_contour = False 
age = 'adult' 
num_betas = 10
num_expression_coeffs = 10 
batch_size = 1 
ext = 'npz' 
sample_shape=True 
sample_expression=True

model1 = smplx.create(model_folder, model_type=model_type,
            gender=gender, use_face_contour=use_face_contour,
            num_betas=num_betas, num_expression_coeffs=num_expression_coeffs,
            ext=ext)  

print(model1)   

SMPLX(
  Gender: NEUTRAL
  Number of joints: 55
  Betas: 10
  Number of PCA components: 6
  Flat hand mean: False
  Number of Expression Coefficients: 10
  (vertex_joint_selector): VertexJointSelector()
)


In [3]:
def load_smplx_data(npz_path): 
    
    data = np.load(npz_path)
    
    global_orient = torch.from_numpy(data['global_orient']).reshape(1, -1)  
    body_pose = torch.from_numpy(data['body_pose']).reshape(1,-1)
    left_hand_pose = torch.from_numpy(data['left_hand_pose']).reshape(1,-1)
    right_hand_pose = torch.from_numpy(data['right_hand_pose']).reshape(1,-1)
    jaw_pose = torch.from_numpy(data['jaw_pose']).reshape(1,-1)
    leye_pose = torch.from_numpy(data['leye_pose']).reshape(1,-1)
    reye_pose = torch.from_numpy(data['reye_pose']).reshape(1,-1) 
    betas = torch.from_numpy(data['betas']).reshape(1,-1)   
    expression = torch.from_numpy(data['expression']).reshape(1,-1) 
    transl = torch.from_numpy(data['transl']).reshape(1,-1)
    
    return {
        'global_orient': global_orient,
        'body_pose': body_pose,
        'left_hand_pose': left_hand_pose,
        'right_hand_pose': right_hand_pose,
        'jaw_pose': jaw_pose,
        'leye_pose': leye_pose,
        'reye_pose': reye_pose,
        'betas': betas,
        'expression': expression,
        'transl': transl
    }  

# def look_at(eye, target, up):
#     f = np.array(target) - np.array(eye)
#     f = f / np.linalg.norm(f)
#     u = np.array(up)
#     s = np.cross(f, u)
#     s = s / np.linalg.norm(s)
#     u = np.cross(s, f)
    
#     m = np.eye(4)
#     m[0, :3] = s
#     m[1, :3] = u
#     m[2, :3] = -f
#     m[:3, 3] = -m[:3, :3] @ eye

#     return m 

npz_file_path = '/Users/lxy/Desktop/yxl/results/demo00001/smplx/00001_0.npz' 
data = load_smplx_data(npz_file_path)   

In [5]:
model_folder = f"/Users/lxy/Gitlab/smplx/models/smplx/SMPLX_NEUTRAL.npz"  
model_type = 'smplx'
gender = 'neutral'
use_face_contour = False 
age = 'adult' 
num_betas = 10
num_expression_coeffs = 10 
batch_size = 1 
ext = 'npz'

sample_shape = None 
sample_expression = None 

plotting_module = 'pyrender'  
plot_joints = False  

import os
import numpy as np
import imageio
import pyrender
import trimesh

base_directory = '/Users/lxy/Desktop/yxl/results/demo00001'
images_directory = os.path.join(base_directory, 'images')

# 确保图像保存的目录存在
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

for root, dirs, files in os.walk(base_directory):
    if 'smplx' in dirs:
        smplx_path = os.path.join(root, 'smplx')
        for file in os.listdir(smplx_path):
            if file.endswith('.npz'):
                npz_file_path = os.path.join(smplx_path, file)
                print("Found npz file:", npz_file_path)  

                data = load_smplx_data(npz_file_path)   
                model = smplx.create(model_folder
                                    , model_type=model_type
                                    , gender=gender  
                                    , global_orient=data['global_orient']
                                    , body_pose=data['body_pose']
                                    , leye_pose=data['leye_pose']
                                    , reye_pose=data['reye_pose'] 
                                    , left_hand_pose=data['left_hand_pose'] 
                                    , right_hand_pose=data['right_hand_pose'] 
                                    , jaw_pose=data['jaw_pose']
                                    , transl=data['transl'] 
                                    , use_pca=False
                                    , use_face_contour=use_face_contour
                                    , num_betas=num_betas
                                    , num_expression_coeffs=num_expression_coeffs
                                    , flat_hand_mean=False
                                    , ext=ext) 

                output = model(global_orient=data['global_orient'], 
                                betas=data['betas'],
                                expression=data['expression'], return_verts=True)  

                vertices = output.vertices.detach().cpu().numpy().squeeze()
                joints = output.joints.detach().cpu().numpy().squeeze()

                print('Vertices shape =', vertices.shape)
                print('Joints shape =', joints.shape)

                vertex_colors = np.ones([vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
                tri_mesh = trimesh.Trimesh(vertices, model.faces, vertex_colors=vertex_colors)
                mesh = pyrender.Mesh.from_trimesh(tri_mesh)
                scene = pyrender.Scene()
                scene.add(mesh)

                if plot_joints:
                    sm = trimesh.creation.uv_sphere(radius=0.005)
                    sm.visual.vertex_colors = [0.9, 0.1, 0.1, 1.0]
                    tfs = np.tile(np.eye(4), (len(joints), 1, 1))
                    tfs[:, :3, 3] = joints
                    joints_pcl = pyrender.Mesh.from_trimesh(sm, poses=tfs)
                    scene.add(joints_pcl)  

                # 创建相机
                camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0) 
                camera_pose = np.array([
                    [1,     0,     0,      0],
                    [0,    -1,     0,      0],
                    [0,     0,     1,     20],
                    [0,     0,     0,    1.0]
                ]) 
                # scene.add(camera, pose=camera_pose)  
                # pyrender.Viewer(scene, use_raymond_lighting=True)

                camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
                camera_pose = np.array([
                    [1.0, 0.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0, -1.0],
                    [0.0, 0.0, 1.0, 100],
                    [0.0, 0.0, 0.0, 1.0]
                ])  
                scene.add(camera, pose=camera_pose)      
                
                pyrender.Viewer(scene, use_raymond_lighting=True) 
            
                # r = pyrender.OffscreenRenderer(1280, 720)
                # color, depth = r.render(scene)  
                # plt.imshow(color)
                # r.delete() 

                # # 构建保存图像的文件名
                # file_name = os.path.join(images_directory, f'{os.path.splitext(file)[0]}.png') 
                
                # # 保存图像
                # imageio.imwrite(file_name, color) 


Found npz file: /Users/lxy/Desktop/yxl/results/demo00001/smplx/00402_0.npz


Vertices shape = (10475, 3)
Joints shape = (127, 3)
Found npz file: /Users/lxy/Desktop/yxl/results/demo00001/smplx/00890_0.npz


KeyboardInterrupt: 

In [7]:
npz_file_path = '/Users/lxy/Desktop/yxl/results/demo00001/smplx/00001_0.npz' 
data = load_smplx_data(npz_file_path)  
  
  
model_folder = f"/Users/lxy/Gitlab/smplx/models/smplx/SMPLX_NEUTRAL.npz"  
model_type = 'smplx'
gender = 'neutral'
use_face_contour = False 
age = 'adult' 
num_betas = 10
num_expression_coeffs = 10 
batch_size = 1 
ext = 'npz'

sample_shape = None 
sample_expression = None 

plotting_module = 'pyrender'  
plot_joints = False    

sample_shape = None 
sample_expression = None 

plotting_module = 'pyrender'  
plot_joints = False

model = smplx.create(model_folder
                     , model_type=model_type
                     , gender=gender  
                     , global_orient=data['global_orient']
                     , body_pose=data['body_pose']
                     , leye_pose=data['leye_pose']
                     , reye_pose=data['reye_pose'] 
                     , left_hand_pose=data['left_hand_pose']
                     , right_hand_pose=data['right_hand_pose'] 
                     , jaw_pose=data['jaw_pose'] 
                     , transl=data['transl']  
                     , use_pca=False
                     , use_face_contour=use_face_contour
                     , num_betas=num_betas
                     , num_expression_coeffs=num_expression_coeffs
                     , flat_hand_mean=False
                     , ext=ext) 

output = model(global_orient=data['global_orient'], 
                betas=data['betas'],
                expression=data['expression'], return_verts=True)  

vertices = output.vertices.detach().cpu().numpy().squeeze()
joints = output.joints.detach().cpu().numpy().squeeze()

print('Vertices shape =', vertices.shape)
print('Joints shape =', joints.shape)

if plotting_module == 'pyrender':
    import pyrender
    import trimesh 

    vertex_colors = np.ones([vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
    tri_mesh = trimesh.Trimesh(vertices, model.faces,
                                vertex_colors=vertex_colors)

    mesh = pyrender.Mesh.from_trimesh(tri_mesh)

    scene = pyrender.Scene()
    scene.add(mesh)

    if plot_joints:
        sm = trimesh.creation.uv_sphere(radius=0.005)
        sm.visual.vertex_colors = [0.9, 0.1, 0.1, 1.0]
        tfs = np.tile(np.eye(4), (len(joints), 1, 1))
        tfs[:, :3, 3] = joints
        joints_pcl = pyrender.Mesh.from_trimesh(sm, poses=tfs)
        scene.add(joints_pcl) 
        
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    camera_pose = np.array([
                    [1,     0,     0,      0],
                    [0,    -1,     0,      0],
                    [0,     0,     1,     20],
                    [0,     0,     0,    1.0]
                ]) 
         
    pyrender.Viewer(scene, use_raymond_lighting=True) 
    


/opt/miniconda3/envs/gui/lib/python3.9/site-packages/smplx/body_models.py:232: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  default_transl = torch.tensor(transl, dtype=dtype)
/opt/miniconda3/envs/gui/lib/python3.9/site-packages/smplx/body_models.py:636: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  default_lhand_pose = torch.tensor(left_hand_pose, dtype=dtype)
/opt/miniconda3/envs/gui/lib/python3.9/site-packages/smplx/body_models.py:648: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  default_rhand_pose = torch.tensor(right_hand_pose, dtyp

Vertices shape = (10475, 3)
Joints shape = (127, 3)


In [10]:
npz_file_path = '/Volumes/Work/Chinese_CV/CCM/Papers/SMPLer-X/demo/results/demo00001/smplx/00034_0.npz'  
model_folder = '/Volumes/Work/Chinese_CV/CCM/Papers/smplx/models/smplx/SMPLX_NEUTRAL.npz'    
data = load_smplx_data(npz_file_path) 

base_directory = '/Volumes/Work/Chinese_CV/CCM/Papers/SMPLer-X/demo/results/demo00001' 
images_directory = os.path.join(base_directory, 'images') 

model_type = 'smplx'
gender = 'neutral'
use_face_contour = False 
age = 'adult' 
num_betas = 10
num_expression_coeffs = 10 
batch_size = 1 
ext = 'npz'

sample_shape = None 
sample_expression = None 

plotting_module = 'pyrender'  
plot_joints = False  

data = load_smplx_data(npz_file_path)  
model = smplx.create(model_folder
                     , model_type=model_type
                     , gender=gender  
                     , global_orient=data['global_orient']
                     , body_pose=data['body_pose']
                     , leye_pose=data['leye_pose']
                     , reye_pose=data['reye_pose']  
                     , left_hand_pose=data['left_hand_pose']
                     , right_hand_pose=data['right_hand_pose']
                     , jaw_pose=data['jaw_pose']
                     , transl=data['transl'] 
                     , use_pca=False
                     , use_face_contour=use_face_contour
                     , num_betas=num_betas
                     , num_expression_coeffs=num_expression_coeffs
                     , flat_hand_mean=False
                     , ext=ext) 

output = model(global_orient=data['global_orient'], 
                betas=data['betas'],
                expression=data['expression'], return_verts=True)  

vertices = output.vertices.detach().cpu().numpy().squeeze()
joints = output.joints.detach().cpu().numpy().squeeze()

print('Vertices shape =', vertices.shape)
print('Joints shape =', joints.shape)

if plotting_module == 'pyrender':
    import pyrender
    import trimesh 

    vertex_colors = np.ones([vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
    tri_mesh = trimesh.Trimesh(vertices, model.faces,
                                vertex_colors=vertex_colors)

    mesh = pyrender.Mesh.from_trimesh(tri_mesh)

    scene = pyrender.Scene()
    scene.add(mesh)

    if plot_joints:  
        sm = trimesh.creation.uv_sphere(radius=0.005)
        sm.visual.vertex_colors = [0.9, 0.1, 0.1, 1.0]
        tfs = np.tile(np.eye(4), (len(joints), 1, 1))
        tfs[:, :3, 3] = joints
        joints_pcl = pyrender.Mesh.from_trimesh(sm, poses=tfs)
        scene.add(joints_pcl) 

    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0 )
    camera_node = pyrender.Node(camera=camera)  
    scene.add_node(camera_node)  

    camera_pose = np.array([
        [1,     0,     0,      0],
        [0,    -1,     0,      -1],
        [0,     0,     1,     28],
        [0,     0,     0,    1.0]
    ]) 

    # 设置相机的位置和方向 
    scene.set_pose(camera_node, pose=camera_pose) 
    scene.add(camera, pose=camera_pose)  

    pyrender.Viewer(scene, use_raymond_lighting=True) 

    # Print camera matrix
    print("Camera pose:\n", camera_pose)

    # r = pyrender.OffscreenRenderer(1280, 720)
    # color, depth = r.render(scene) 
    # plt.figure(dpi=300)
    # plt.axis('off')
    # plt.imshow(color) 

    # r = pyrender.OffscreenRenderer(1280, 720) 
    # color, depth = r.render(scene)   
    # plt.figure(figsize=(18,12))    
    # plt.imshow(color)  
    # plt.axis('off')
    # r.delete() 

    # 构建保存图像的文件名
    # file_name = os.path.join(images_directory, f'{os.path.splitext(file)[0]}.png') 
    
    # 保存图像 
    # imageio.imwrite(file_name, color)  

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Work/Chinese_CV/CCM/Papers/SMPLer-X/demo/results/demo00001/smplx/00034_0.npz'

: 